In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
''''
Labels are:
0 - Angry
1 - Disgust
2 - Fear
3 - Happy
4 - Sad
5 - Surprise
6 - Neutral
'''

##########################################lk

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import required libraries
import tensorflow as tf
import cv2
import os
import numpy as np
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization, Dropout

# Define paths for training, validation, and test data
train_directory = "/content/drive/MyDrive/Vit/Semester 3/Deep learning /deep learning lk data/data/train"
val_directory = "/content/drive/MyDrive/Vit/Semester 3/Deep learning /deep learning lk data/data/val"
test_directory = "/content/drive/MyDrive/Vit/Semester 3/Deep learning /deep learning lk data/data/test"

# Define class names
Classes = ["0", "1", "2", "3", "4", "5", "6"]
num_classes = len(Classes)
img_size = 224  # Target image size for resizing

# Function to load and preprocess images from a given directory
# Function to load and preprocess images from a given directory
def load_data(directory):
    data = []
    for category in Classes:
        path = os.path.join(directory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)  # Convert image to RGB
                img_array = cv2.resize(img_array, (img_size, img_size))  # Resize image
                data.append([img_array, class_num])
            except Exception as e:
                pass
    return data




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load and prepare training and validation data
training_data = load_data(train_directory)
validation_data = load_data(val_directory)

# Shuffle and separate features and labels for training data
random.shuffle(training_data)
X_train = np.array([item[0] for item in training_data]).reshape(-1, img_size, img_size, 3) / 255.0
y_train = np.array([item[1] for item in training_data])

# Separate features and labels for validation data
X_val = np.array([item[0] for item in validation_data]).reshape(-1, img_size, img_size, 3) / 255.0
y_val = np.array([item[1] for item in validation_data])

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)



In [ ]:
# Image augmentation for training data
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
datagen.fit(X_train)  # Apply augmentation to training data

# Load MobileNetV2 with pretrained weights
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    input_shape=(img_size, img_size, 3),
    pooling='avg'
)
base_model.trainable = False  # Freeze all layers initially

# Unfreeze the last few layers (e.g., last 30 layers) for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Add new layers on top of the base model
base_input = base_model.input
base_output = base_model.output
final_output = tf.keras.layers.Dense(128, activation='relu')(base_output)
final_output = BatchNormalization()(final_output)
final_output = Dropout(0.5)(final_output)
final_output = tf.keras.layers.Dense(64, activation='relu')(final_output)
final_output = BatchNormalization()(final_output)
final_output = Dropout(0.5)(final_output)
final_output = tf.keras.layers.Dense(len(Classes), activation='softmax')(final_output)

# Create the new model
new_model = tf.keras.Model(inputs=base_input, outputs=final_output)
new_model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [ ]:
# Compile the model
optimizer = Adam(learning_rate=0.0001)
new_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate reduction callback
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)



In [ ]:
# Prepare data generators for training and validation
batch_size = 32
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = ImageDataGenerator().flow(X_val, y_val, batch_size=batch_size, shuffle=False)

# Train the model
new_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=40,
    callbacks=[lr_callback]
)



Epoch 1/40
262/262 [==============================] - 129s 474ms/step - loss: 2.7633 - accuracy: 0.2037 - val_loss: 1.9362 - val_accuracy: 0.3143 - lr: 1.0000e-04
Epoch 2/40
262/262 [==============================] - 121s 463ms/step - loss: 2.3717 - accuracy: 0.2746 - val_loss: 1.9162 - val_accuracy: 0.3710 - lr: 1.0000e-04
Epoch 3/40
262/262 [==============================] - 122s 466ms/step - loss: 2.2157 - accuracy: 0.3004 - val_loss: 2.0313 - val_accuracy: 0.3762 - lr: 1.0000e-04
Epoch 4/40
262/262 [==============================] - 120s 457ms/step - loss: 2.0275 - accuracy: 0.3398 - val_loss: 1.8571 - val_accuracy: 0.4256 - lr: 1.0000e-04
Epoch 5/40
262/262 [==============================] - 121s 462ms/step - loss: 1.8962 - accuracy: 0.3643 - val_loss: 1.6568 - val_accuracy: 0.4504 - lr: 1.0000e-04
Epoch 6/40
262/262 [==============================] - 121s 460ms/step - loss: 1.7937 - accuracy: 0.3834 - val_loss: 1.5302 - val_accuracy: 0.4602 - lr: 1.0000e-04
Epoch 7/40
262/262 [==

In [ ]:
# Save the trained model
new_model.save("/content/drive/MyDrive/Deep learning project/new_model.h5")



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load and prepare test data
test_data = load_data(test_directory)
X_test = np.array([item[0] for item in test_data]).reshape(-1, img_size, img_size, 3) / 255.0
y_test = np.array([item[1] for item in test_data])

# Convert test labels to one-hot encoding
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)




In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = new_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.2f}')
print(f'Test Loss: {test_loss:.2f}')

15/15 [==============================] - 3s 203ms/step - loss: 1.0048 - accuracy: 0.6404
Test Accuracy: 0.64
Test Loss: 1.00


########################additional dropout and L2 regularization, more data augmentation, learning rate adjustment, and early stopping.

In [ ]:
# Import required libraries
import tensorflow as tf
import cv2
import os
import numpy as np
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2

# Define paths for training, validation, and test data
train_directory = "/content/drive/MyDrive/Vit/Semester 3/Deep learning /deep learning lk data/data/train"
val_directory = "/content/drive/MyDrive/Vit/Semester 3/Deep learning /deep learning lk data/data/val"
test_directory = "/content/drive/MyDrive/Vit/Semester 3/Deep learning /deep learning lk data/data/test"

# Define class names
Classes = ["0", "1", "2", "3", "4", "5", "6"]  # Ensure these match folder names exactly
num_classes = len(Classes)
img_size = 224  # Target image size for resizing

# Function to load and preprocess images from a given directory
def load_data(directory):
    data = []
    for category in Classes:
        path = os.path.join(directory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img_array = cv2.resize(img_array, (img_size, img_size))  # Resize image
                data.append([img_array, class_num])
            except Exception as e:
                pass
    return data



In [ ]:
# Load and prepare training and validation data
training_data = load_data(train_directory)
validation_data = load_data(val_directory)

# Shuffle and separate features and labels for training data
random.shuffle(training_data)
X_train = np.array([item[0] for item in training_data]).reshape(-1, img_size, img_size, 3) / 255.0
y_train = np.array([item[1] for item in training_data])

# Separate features and labels for validation data
X_val = np.array([item[0] for item in validation_data]).reshape(-1, img_size, img_size, 3) / 255.0
y_val = np.array([item[1] for item in validation_data])

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)



In [ ]:
# Image augmentation for training data
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # New brightness range
    shear_range=0.15,             # New shear range
    fill_mode="nearest"
)
datagen.fit(X_train)  # Apply augmentation to training data

# Load MobileNetV2 with pretrained weights
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    input_shape=(img_size, img_size, 3),
    pooling='avg'
)
base_model.trainable = False  # Freeze all layers initially

# Unfreeze the last few layers for fine-tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Add new layers on top of the base model with adjusted Dropout, L2 regularization, and units
base_input = base_model.input
base_output = base_model.output
base_output = BatchNormalization()(base_output)  # Added batch normalization here
final_output = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.0001))(base_output)
final_output = BatchNormalization()(final_output)
final_output = Dropout(0.4)(final_output)  # Reduced dropout rate
final_output = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.0001))(final_output)
final_output = BatchNormalization()(final_output)
final_output = Dropout(0.4)(final_output)
final_output = tf.keras.layers.Dense(len(Classes), activation='softmax')(final_output)


# Create the new model
new_model = tf.keras.Model(inputs=base_input, outputs=final_output)
new_model.summary()



Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_5[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']      

In [ ]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
new_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for learning rate adjustment and early stopping
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Prepare data generators for training and validation
batch_size = 32
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = ImageDataGenerator().flow(X_val, y_val, batch_size=batch_size, shuffle=False)



In [ ]:
# Train the model with callbacks
new_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=60,  # Set a higher epoch count but let early stopping manage it
    callbacks=[lr_callback, early_stopping_callback]
)



Epoch 1/60
262/262 [==============================] - 150s 548ms/step - loss: 2.3486 - accuracy: 0.1561 - val_loss: 3.0715 - val_accuracy: 0.1504 - lr: 0.0010
Epoch 2/60
262/262 [==============================] - 142s 540ms/step - loss: 2.0828 - accuracy: 0.1765 - val_loss: 2.1762 - val_accuracy: 0.1265 - lr: 0.0010
Epoch 3/60
262/262 [==============================] - 141s 537ms/step - loss: 2.0125 - accuracy: 0.1714 - val_loss: 2.1350 - val_accuracy: 0.1072 - lr: 0.0010
Epoch 4/60
262/262 [==============================] - 142s 541ms/step - loss: 1.9658 - accuracy: 0.1716 - val_loss: 2.1586 - val_accuracy: 0.1072 - lr: 0.0010
Epoch 5/60
262/262 [==============================] - 142s 542ms/step - loss: 1.9532 - accuracy: 0.1759 - val_loss: 2.5661 - val_accuracy: 0.1422 - lr: 0.0010
Epoch 6/60
262/262 [==============================] - 142s 542ms/step - loss: 1.9346 - accuracy: 0.1822 - val_loss: 2.3470 - val_accuracy: 0.1072 - lr: 0.0010
Epoch 7/60
262/262 [==========================

In [ ]:
# Save the trained model
new_model.save("/content/drive/MyDrive/Deep learning project/new_model.h5")




In [ ]:
# Load and prepare test data
test_data = load_data(test_directory)
X_test = np.array([item[0] for item in test_data]).reshape(-1, img_size, img_size, 3) / 255.0
y_test = np.array([item[1] for item in test_data])

# Convert test labels to one-hot encoding
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)



In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = new_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.2f}')
print(f'Test Loss: {test_loss:.2f}')

15/15 [==============================] - 3s 188ms/step - loss: 3.3530 - accuracy: 0.1511
Test Accuracy: 0.15
Test Loss: 3.35
